In [ ]:
import os
import sys
from transformers import BertTokenizer, BertModel
import torch


## This is a dummy example, full example is after this cell

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
def get_bert_embeddings(sentence_a, sentence_b):
    # Encode the input sentences
    inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt', max_length=100, truncation=True, padding='max_length')
    #print ("inputs: ",inputs)

    # Get token, segment (token_type_ids), and position embeddings
    with torch.no_grad():
        outputs = model(**inputs)

    # Token embeddings for the entire sequence
    token_embeddings = outputs.last_hidden_state
    return token_embeddings


# Sample sentences
sentence_a = "The stock market is rising."
sentence_b = "Investors are optimistic."

embeddings = get_bert_embeddings(sentence_a, sentence_b)
print(embeddings)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tensor([[[-0.4286,  0.1137, -0.2656,  ..., -0.7443,  0.4859,  0.2045],
         [-0.3168, -0.5687, -0.6849,  ..., -0.0050,  1.1457, -0.3311],
         [ 1.1782, -0.8121,  0.2693,  ..., -0.5846,  0.5393, -0.5659],
         ...,
         [ 0.5979, -0.0870,  0.4546,  ..., -0.1556,  0.0917, -0.3057],
         [ 0.5636, -0.1026,  0.4577,  ..., -0.1923,  0.1060, -0.3252],
         [ 0.0797, -0.5168,  0.2247,  ...,  0.0028,  0.1853, -0.6546]]])


## This is the full example to find the corpus which is close to given text

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# Load pre-trained FinBERT model and tokenizer
#model_name = "bert-base-uncased"
model_name = "yiyanghkust/finbert-tone"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [ ]:

# Sample corpus
corpus = [
    "The stock market is bullish.",
    "Investors are optimistic",
    "The trend might continue.",
    "Investing in technology stocks is risky.",
    "The financial report was positive.",
    "The economic outlook is uncertain."
]
corpus


['The stock market is bullish.',
 'Investors are optimistic',
 'The trend might continue.',
 'Investing in technology stocks is risky.',
 'The financial report was positive.',
 'The economic outlook is uncertain.']

In [ ]:

# Tokenize and compute embeddings for the corpus
inputs = tokenizer(corpus, padding=True, truncation=True, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
    corpus_embeddings = outputs.last_hidden_state[:, 0, :].numpy()  # CLS token embeddings

# Store the corpus and embeddings in a DataFrame for easy lookup
corpus_df = pd.DataFrame({
    'text': corpus,
    'embedding': list(corpus_embeddings)
})
corpus_df

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,text,embedding
0,The stock market is bullish.,"[0.3219699, -0.5775221, 0.18884265, -1.1663916..."
1,Investors are optimistic,"[-0.15846898, -0.71165615, -0.53034717, -0.672..."
2,The trend might continue.,"[-0.77168113, -0.25501037, -1.6606758, 0.70160..."
3,Investing in technology stocks is risky.,"[-1.859709, -0.31047294, -0.015470292, -0.5514..."
4,The financial report was positive.,"[0.29098234, -0.24167642, 0.22606759, -1.20790..."
5,The economic outlook is uncertain.,"[-1.7298591, -1.7114106, -0.73854214, 0.701667..."


In [ ]:

# Select a text from the corpus to compute its embedding
selected_text = "The stock market is bearish."
inputs = tokenizer(selected_text, return_tensors="pt")

# Compute the embedding for the selected text
with torch.no_grad():
    outputs = model(**inputs)
    new_embedding = outputs.last_hidden_state[:, 0, :].numpy()  # CLS token embedding
new_embedding.shape


(1, 768)

In [ ]:
# Compute cosine similarity between the new embedding and corpus embeddings
similarities = cosine_similarity(new_embedding, corpus_embeddings)
similarities


array([[0.38055733, 0.37139723, 0.49765167, 0.4900169 , 0.22699091,
        0.59797335]], dtype=float32)

In [ ]:
not
# Find the index of the most similar text
most_similar_index = np.argmax(similarities)

# Retrieve the most similar text
most_similar_text = corpus_df.iloc[most_similar_index]['text']
print("Original text:", selected_text)
print("Most similar text:", most_similar_text)


SyntaxError: invalid syntax (<ipython-input-104-324e2158bc99>, line 1)